<a href="https://colab.research.google.com/github/woraphonp-038-5/Project_LineChatbot_Contraception/blob/main/Code_%E0%B8%96%E0%B8%AD%E0%B8%94%E0%B8%84%E0%B8%A7%E0%B8%B2%E0%B8%A1_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ถอดความ**

In [ ]:
!pip install pdf2docx pymupdf pdfplumber pytesseract pdf2image python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 121.0 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,858 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:14 ht

In [ ]:
import fitz
import pdfplumber
from pdf2docx import Converter
from docx import Document
from pdf2image import convert_from_path
import pytesseract
import re

#-------------------------------------------------------------------------------
PDF_FILE = "/content/WHO MEC 2025 6th edition.pdf"
OUTPUT_DOCX = "WHO MEC 2025 6th edition 1-10.docx"

START_PAGE = 1     # <<< เลือกหน้าเริ่ม
END_PAGE = 30      # <<< เลือกหน้าสุดท้าย
#-------------------------------------------------------------------------------

ILLEGAL_XML_CHARS_PATTERN = re.compile(
    r"[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]"
)

def clean_xml_text(text):
    return ILLEGAL_XML_CHARS_PATTERN.sub("", text or "")

# ---------- ตรวจว่าเป็น text-based ----------
def has_text(pdf_path, start, end):
    with fitz.open(pdf_path) as doc:
        for i in range(start - 1, min(end, len(doc))):
            if doc[i].get_text().strip():
                return True
    return False

# ---------- pdf2docx ----------
def convert_with_pdf2docx(pdf_path, docx_path, start, end):
    cv = Converter(pdf_path)
    cv.convert(
        docx_path,
        start=start - 1,
        end=end
    )
    cv.close()

# ---------- ดึงเฉพาะตาราง ----------
def extract_tables(pdf_path, start, end):
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start - 1, min(end, len(pdf.pages))):
            for table in pdf.pages[i].extract_tables():
                for row in table:
                    rows.append("\t".join(cell or "" for cell in row))
                rows.append("")
    return clean_xml_text("\n".join(rows))

# ---------- OCR ----------
def extract_text_ocr(pdf_path, start, end):
    texts = []
    images = convert_from_path(
        pdf_path,
        dpi=200,
        first_page=start,
        last_page=end
    )
    for img in images:
        text = pytesseract.image_to_string(img, lang="eng")
        texts.append(clean_xml_text(text))
    return "\n".join(texts)

# ---------- MAIN ----------
if has_text(PDF_FILE, START_PAGE, END_PAGE):
    print("Text-based PDF")

    convert_with_pdf2docx(
        PDF_FILE,
        OUTPUT_DOCX,
        START_PAGE,
        END_PAGE
    )

    tables = extract_tables(PDF_FILE, START_PAGE, END_PAGE)

    if tables.strip():
        doc = Document(OUTPUT_DOCX)
        doc.add_page_break()
        doc.add_heading("Extracted Tables", level=1)

        for line in tables.split("\n"):
            doc.add_paragraph(line)

        doc.save(OUTPUT_DOCX)

else:
    print("Scanned PDF → OCR")

    doc = Document()
    doc.add_heading("OCR Result", level=1)

    for para in extract_text_ocr(PDF_FILE, START_PAGE, END_PAGE).split("\n\n"):
        doc.add_paragraph(para)

    doc.save(OUTPUT_DOCX)

print("Finished:", OUTPUT_DOCX)

Text-based PDF
Finished: WHO MEC 2025 6th edition 1-10.docx
